In [ ]:
# !pip install streamlit

In [15]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.retrieval import create_retrieval_chain
from langchain_community.vectorstores import FAISS

In [16]:
import random
from groq import Groq
import streamlit as st
api_keys = ['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','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',
            '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','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']

client = Groq(
    
    api_key = random.choice(api_keys)
)

llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192",temperature=0.0)



In [17]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import os.path
import pickle

# Define the scope
SCOPES = ['https://www.googleapis.com/auth/drive']
credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"
def authenticate_google_drive():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(credentials_path, SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('drive', 'v3', credentials=creds)

service = authenticate_google_drive()


In [18]:
def list_all_files(service):
    all_files = {}
    page_token = None

    while True:
        response = service.files().list(
            pageSize=100,
            fields="nextPageToken, files(id, name, mimeType)",
            pageToken=page_token
        ).execute()

        files = response.get('files', [])
        for file in files:
            all_files[file['name']] = (file['id'], file['mimeType'])

        page_token = response.get('nextPageToken')
        if not page_token:
            break

    return all_files

# Retrieve all files and store them in a dictionary
all_files_dict = list_all_files(service)


In [19]:
len(all_files_dict)

13216

In [21]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Initialize the HuggingFaceEmbeddings with a specific model
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True, 'batch_size': 224}

# Create an instance of HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Prepare file descriptions for embedding
file_descriptions = [f"Name: {name}, ID: {details[0]}, Type: {details[1]}" for name, details in all_files_dict.items()]

# Generate embeddings for the file descriptions
document_embeddings = embeddings.embed_documents(file_descriptions)

# Display the first few embeddings to verify
print(document_embeddings[:5])
